<a href="https://colab.research.google.com/github/Visors/d2l-zh-2/blob/main/tensorflow/chapter_deep-learning-computation/custom-layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。


In [1]:
import tensorflow as tf


class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

让我们向该层提供一些数据，验证它是否能按预期工作。


In [2]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

现在，我们可以[**将层作为组件合并到更复杂的模型中**]。


In [3]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [4]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=-5.587935447692871e-09>

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。


In [6]:
import tensorflow as tf

class MyDense(tf.keras.Model):
    """
    自定义的全连接层 (Dense Layer)

    继承自 tf.keras.Model，用于创建具有权重和偏置的自定义层。
    """
    def __init__(self, units):
        """
        类的构造函数

        Args:
            units (int): 输出单元的数量，即该层的输出维度。
        """
        super().__init__()  # 调用父类 tf.keras.Model 的构造函数
        self.units = units  # 保存输出单元数量，作为该层的属性

    def build(self, X_shape):
        """
        构建层的权重和偏置

        此方法在第一次调用 call 方法时自动调用，用以延迟初始化。
        用于创建层的可训练参数。

        Args:
            X_shape (tuple): 输入张量的形状 (shape)。
        """
        # 添加权重 (weight)
        self.weight = self.add_weight(
            name='weight',  # 参数名称
            shape=[X_shape[-1], self.units],  # 权重矩阵的形状：[输入特征数, 输出单元数]
            initializer=tf.random_normal_initializer()  # 使用正态分布初始化权重
        )
        # 添加偏置 (bias)
        self.bias = self.add_weight(
            name='bias',  # 参数名称
            shape=[self.units],  # 偏置向量的形状：[输出单元数]
            initializer=tf.zeros_initializer()  # 使用零初始化偏置
        )

    def call(self, X):
        """
        定义层的前向传播逻辑

        Args:
            X (tf.Tensor): 输入张量。

        Returns:
            tf.Tensor: 输出张量。
        """
        linear = tf.matmul(X, self.weight) + self.bias  # 线性变换：矩阵乘法 + 偏置
        return tf.nn.relu(linear)  # 应用 ReLU 激活函数

**`tf.keras.Model` 的自动构建机制：**（mio按）

*   当你创建一个继承自 `tf.keras.Model` 的自定义层（例如 `MyDense`）时，你不需要手动去调用 `build` 方法。TensorFlow 会自动帮你完成这个过程。
*   当你第一次用某个输入 `X` 调用这个自定义层的 `call` 方法时（例如 `dense(tf.random.uniform((2, 5)))`），TensorFlow 会做以下事情：
    1.  **检查是否已构建：** 检查这个层是否已经调用过 `build` 方法创建了它的权重和偏置等参数。
    2.  **未构建则调用 `build`：** 如果该层还没有构建过（即第一次调用），TensorFlow 会**自动**调用该层的 `build` 方法。
    3.  **推断 `X_shape`：** 在调用 `build` 方法之前，TensorFlow 会分析你传入 `call` 方法的输入张量 `X` 的形状，即 `X.shape`。
    4.  **传递 `X.shape` 给 `build`：**  TensorFlow 会把 `X.shape` 以某种形式（通常是 `tf.TensorShape` 对象）作为参数 `X_shape` 传递给 `build` 方法。这就是 `build` 方法中 `X_shape` 的来源。
    5.  **`build` 创建参数：**  `build` 方法使用 `X_shape` 来确定需要创建的参数（例如权重矩阵）的形状，然后创建这些参数。
    6. **调用`call`:** 创建参数之后再进行`call`的计算。

**关键点：`tf.keras.Model` 内部的生命周期管理**

`tf.keras.Model` 类（以及其基类 `tf.keras.layers.Layer`）内部实现了一个复杂的生命周期管理机制。这个机制的核心任务是：

1.  **跟踪模型/层是否已经构建：** 判断模型/层的参数是否已经被创建（即 `build` 方法是否执行过）。
2.  **在需要时自动触发构建：** 如果模型/层还没有构建，并且需要使用，则自动调用 `build` 方法。

**详细解释自动执行 `build` 的过程：**

1.  **`__call__` 方法：**
    *   当你调用 `dense(tf.random.uniform((2, 5)))` 时，你实际上是在调用 `MyDense` 类（或者任何继承自 `tf.keras.Model` 的类的）的 `__call__` 方法。
    *   请注意，是你定义的`call`函数被`__call__`所调用。`__call__`是一个特殊的函数，使得对象可以像函数一样被调用。例如：`dense(tf.random.uniform((2, 5)))`。
    *   这个 `__call__` 方法是 `tf.keras.Model`（或 `tf.keras.layers.Layer`）中定义的，而不是你在 `MyDense` 类中定义的。
    *   这个 `__call__` 方法是**关键所在**，因为它负责在真正执行你写的 `call` 方法之前，进行一些重要的检查和操作。

2.  **`__call__` 方法内部的检查和构建：**
    *   `__call__` 方法首先会检查这个模型/层是否已经构建了，这个检查一般是通过一个内部的标志位来实现的（例如 `self.built`）。
    *   如果模型/层还没有构建（`self.built` 为 `False`），`__call__` 方法就会：
        *   **推断输入形状：** 根据你传递给它的输入张量 `X`，推断出输入数据的形状，得到 `X.shape`。
        *   **调用 `build` 方法：** 调用你定义的 `build` 方法，并将推断出的输入形状作为 `X_shape` 参数传递给 `build` 方法。
        * **将 `self.built`设置为`True`**: 将模型内的`self.built`标志设置为`True`，表示此模型已经被构建。
    *   如果模型/层已经构建（`self.built` 为 `True`），`__call__` 方法就会跳过构建步骤，直接进入下一步。

3.  **执行 `call` 方法：**
    *   无论模型/层是否需要构建，`__call__` 方法的最后一步都是调用你定义的 `call` 方法，并将输入张量 `X` 传递给 `call` 方法。
    *   现在，因为已经确保模型/层是构建完成的，所以你的 `call` 方法可以安全地访问和使用模型的参数（`self.weight` 和 `self.bias`）。

**总结：**

*   **`tf.keras.Model` 的 `__call__` 方法是核心：**  它负责管理模型/层的生命周期，包括检查是否已构建，以及在需要时自动调用 `build` 方法。
* **重写的是`call`**: 我们定义的`call`函数，是对tf.keras.Model父类中定义好的`__call__`方法所使用的函数体的重写。我们并没有直接重写`__call__`函数。
*   **`build` 方法的自动执行：**  正是因为 `tf.keras.Model` 的 `__call__` 方法中的这些检查和调用机制，我们才能在第一次调用模型/层时，自动触发 `build` 方法的执行，而无需我们手动调用。
*   **延迟初始化：**  这是一种延迟初始化的技术，参数的创建被推迟到了第一次需要使用这些参数的时候。

**为什么要有这个机制？**

这种自动构建机制极大地简化了模型/层的定义和使用。你不需要关心何时去调用 `build` 方法，`tf.keras.Model` 会帮你处理好这些细节。 你只需要写好 `__init__`，`build`和`call`这三个方法就可以了。

接下来，我们实例化`MyDense`类并访问其模型参数。


In [17]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.03281968, -0.00808049,  0.03600667],
        [ 0.10606171,  0.0393287 , -0.04071729],
        [ 0.07607688,  0.02219767,  0.07790767],
        [-0.04000024, -0.02157551,  0.11163219],
        [-0.03728421,  0.01720558, -0.03382197]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

我们可以[**使用自定义层直接执行前向传播计算**]。


In [18]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.08145942, 0.03432899, 0.06407103],
       [0.10259131, 0.05410828, 0.05708903]], dtype=float32)>

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [19]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00870765],
       [0.        ]], dtype=float32)>

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。


In [43]:
# 练习 1
class TensorReductionLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        # 输入形状: (batch_size, n)
        n = input_shape[-1]
        # 添加权重: (n, n, units)
        self.W = self.add_weight(
            name='W',
            shape=(n, n, self.units),
            initializer=tf.keras.initializers.GlorotNormal()
        )

    def call(self, X):
        # 使用爱因斯坦求和替代循环
        outer_product = tf.einsum("bi,bj->bij", X, X)
        y = tf.einsum("bij,ijk->bk", outer_product, self.W)
        return y

In [44]:
# 练习 1 测试
net = tf.keras.Sequential([TensorReductionLayer(4)])
test_input = tf.random.uniform((2, 5))
output = net(test_input)
print(output.numpy)
print(output.shape)

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.17281182,  0.2763521 ,  0.27563778, -0.05836287],
       [ 0.08673773, -0.4523106 ,  0.6179464 , -0.21719313]],
      dtype=float32)>>
(2, 4)


[Discussions](https://discuss.d2l.ai/t/1836)
